In [1]:
import os
import sys
import pandas as pd

import skops.io as sio
import base64
import logging

sys.path.append(os.path.join(os.getcwd(), '..'))

from schemas.model_selection import ModelSelectionRequest
from agents.model_selection_agent import run_model_agent
from schemas.feature import FeatureSpec

In [2]:
selected_features = [FeatureSpec(name='Fare', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.25), FeatureSpec(name='SibSp', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.15), FeatureSpec(name='Pclass', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.1), FeatureSpec(name='Parch', dtype='numeric', origin='raw', transformer='StandardScaler', params={}, importance=0.05)]
preprocessing_code = '...'

In [3]:
selected_feature_names = [feature.name for feature in selected_features]

selected_feature_names

['Fare', 'SibSp', 'Pclass', 'Parch']

In [4]:
def decode_pipeline(base64_blob):
    binary_blob = base64.b64decode(base64_blob)
    pipe = sio.loads(binary_blob)
    return pipe

In [5]:
decoded_pipeline = decode_pipeline(preprocessing_code)

str_decoded_pipeline = str(decoded_pipeline)

# drop blank lines from the pipeline
str_decoded_pipeline = "\n".join([line for line in str_decoded_pipeline.split("\n") if line.strip() != ""])

str_decoded_pipeline

"ColumnTransformer(transformers=[('num',\n                                 Pipeline(steps=[('impute',\n                                                  SimpleImputer(strategy='median')),\n                                                 ('scale', StandardScaler())]),\n                                 ['Fare', 'SibSp', 'Pclass', 'Parch']),\n                                ('cat',\n                                 Pipeline(steps=[('impute',\n                                                  SimpleImputer(strategy='most_frequent')),\n                                                 ('encode',\n                                                  OneHotEncoder(handle_unknown='ignore'))]),\n                                 [])])"

In [6]:
df = pd.read_csv("../datasets/titanic.csv").head(10)
data_sample = df.to_dict(orient="list")

In [7]:
from schemas.shared import Metadata


req = ModelSelectionRequest(
    metadata=Metadata(
        dataset_name="Titanic",
        problem_type="classification",
        target_column="Survived"
    ),
    data=data_sample,
    selected_features=selected_feature_names,
)

In [8]:
resp = run_model_agent(req, preprocessing_code)

2025-05-28 15:42:02.934 | INFO     | agents.model_selection_agent:run_model_agent:98 - Processing request for dataset 'Titanic'
2025-05-28 15:42:02.935 | INFO     | agents.model_selection_agent:_build_prompt:52 - Building the prompt for model selection.
2025-05-28 15:42:02.935 | INFO     | agents.model_selection_agent:run_model_agent:102 - Prompt length: 154 characters
2025-05-28 15:42:13.851 | INFO     | agents.model_selection_agent:run_model_agent:107 - LLM response received successfully
2025-05-28 15:42:13.880 | INFO     | agents.model_selection_agent:run_model_agent:113 - Pipeline serialized to {len(pipe_blob)} bytes
2025-05-28 15:42:13.881 | INFO     | agents.model_selection_agent:run_model_agent:119 - Model selection completed successfully


In [ ]:
resp[0]
resp[1]

'...'

In [12]:
df = pd.read_csv("../datasets/titanic.csv")

pipeline = decode_pipeline(resp[1])

X = df.drop(columns=["Survived"])
y = df["Survived"]

pipeline.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Fare', 'SibSp', 'Pclass',
                                                   'Parch']),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', RandomForestClassifier(max_depth=5))])

In [14]:
y_pred = pipeline.predict(X)

# calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy of the model: {accuracy:.2f}")


Accuracy of the model: 0.76
